Лучший результат на cv: 0.9724

Модель: LinearSVC()

Признаки: 
 - 1535 шт CountVectorizer(analyzer='char_wb', ngram_range=(2, 4)) + отбор ExtraTreesClassifier
 - CountVectorizer(ngram_range=(1, 2)) на тексте, полученном из shape слов (функция shape_word()) 
 - количество предложений в сообщении
 - количество ключевых спам-слов в сообщении ('£', 'http','www.', '.com', 'prize', 'FREE', '18+', ...) 



In [143]:
text_file = open('SMSSpamCollection.txt')
raw_corpus = []
for i, line in enumerate(text_file):
    raw_corpus.append([line.split()[0].strip(), line.split('\t', 1 )[1].strip()])

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import numpy as np
import nltk  

In [145]:
from nltk.stem.porter import *

import string
alp_lower = list(string.ascii_lowercase)
alp_upper = list(string.ascii_uppercase)
numbers = list(string.digits)
punkt=['+', '.', '?', '!', '/', '-', '=', ',', '&', u'£']

stemmer = PorterStemmer()

# переводим строчные буквы в x, заглавные в X, цифры в 0 и схлопываем подряд идущие дубли
def shape_word(word):
    new_word = '1'
    for ch in word:
        if ch in alp_lower and new_word[-1] != 'x':
            new_word += 'x'
        elif ch in alp_lower and new_word[-1] == 'x':
            a=[]
        elif ch in alp_upper and new_word[-1] != 'X':
            new_word += 'X'
        elif ch in alp_upper and new_word[-1] == 'X':
            a=[]
        elif ch in numbers and new_word[-1] != '0':
            new_word +='0'
        elif ch in numbers and new_word[-1] == '0':
            a=[]
        elif ch in punkt:
            new_word += ch
        else:
            new_word += '_'
    return new_word[1:]


def tok(sentense):
    return [shape_word(word)for word in sentense.split()]

#количество цифр в сообещении
def count_number(text):
    count = 0
    for ch in text:
        if ch in numbers:
            count+=1
    return [count]

In [146]:
Y = [raw_corpus[i][0]=='spam' for i in range(len(raw_corpus))]

In [147]:
vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 4))
vectorizer_mask = CountVectorizer(tokenizer=tok, ngram_range=(1, 2))
sent_length = [[len(nltk.sent_tokenize(raw_corpus[i][1].decode('utf-8')))] for i in range(len(raw_corpus))]
X = [vectorizer.fit_transform([raw_corpus[i][1] for i in range(len(raw_corpus))]).toarray(), vectorizer_mask.fit_transform([raw_corpus[i][1] for i in range(len(raw_corpus))]).toarray(), sent_length]

In [148]:
#метки спама
spam_words = ['£', 'http','www.', '.com', 
              'prize', '.net', 'only', 'free', 'now', 'call', 'yes', 'no', 'txt'
              'sms', 'award', 'go', 'stop', '18+', '16+']
X_spam_labels = []
for x in raw_corpus:
    spam_labels = []
    for word in spam_words:
        spam_labels.append(x[1].lower().find(word) != -1)
    X_spam_labels.append([sum(spam_labels)])
X.append(X_spam_labels)

In [149]:
X.append([count_number(raw_corpus[i][1]) for i in range(len(raw_corpus))])

In [150]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizerTfV = TfidfVectorizer()
X.append(vectorizerTfV.fit_transform([raw_corpus[i][1] for i in range(len(raw_corpus))]).toarray())

In [151]:
X.append([count_number(raw_corpus[i][1]) for i in range(len(raw_corpus))])

In [152]:
from sklearn.svm import LinearSVC

In [188]:
clSVC = LinearSVC()
X_test = X[0]
for i in range(1, 6):
    X_test = np.hstack((X_test, X[i]))
    scores = cross_val_score(clSVC, X_test, Y, cv=10, scoring='f1')
    print np.mean(scores)

0.966107861528
0.965426614778
0.966826252333
0.96424601158
0.964906462241


In [156]:
#отбор признаков, полученных TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
ets = ExtraTreesClassifier(n_estimators=70, max_features=4000, bootstrap=True, n_jobs=-1)
X.append(ets.fit_transform(X[5], Y))

/Users/physteshka/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [161]:
#отбор признков, полученных CountVectorizer
from sklearn.ensemble import ExtraTreesClassifier
ets = ExtraTreesClassifier(n_estimators=70, max_features=4000, bootstrap=True, n_jobs=-1)
X.append(ets.fit_transform(X[0], Y))

/Users/physteshka/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [197]:
X_test = X[8]
features_number = [1, 2, 3, 7, 4, 6]
for i in features_number:
    X_test = np.hstack((X_test, X[i]))
    scores = cross_val_score(clSVC, X_test, Y, cv=10, scoring='f1')
    print np.mean(scores)

0.971710466105
0.971710466105
0.972449164027
0.972449164027
0.971889802505
0.971889802505


In [182]:
ets = ExtraTreesClassifier(n_estimators=100, max_features=4000, bootstrap=True, n_jobs=-1)
X.append(ets.fit_transform(X_test, Y))

/Users/physteshka/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [184]:
scores = cross_val_score(clSVC, X[-1], Y, cv=10, scoring='f1')
print np.mean(scores)

0.969000581194


In [185]:
etc = ExtraTreesClassifier(n_estimators=400, bootstrap=True, n_jobs=-1)
scores = cross_val_score(etc, X[-1], Y, cv=10, scoring='f1')
print np.mean(scores)

0.969645000348
